Mount Google Drive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q pandas


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [58]:
import pandas as pd

variant_path = '/content/drive/MyDrive/Data Science Final Project/Data/mutations_cellosaurus_full.csv'
variant_path = 'data/mutations_cellosaurus_full.csv'
variants = pd.read_csv(variant_path)

/var/folders/rk/_8vnc6913c5fyc733rjczgp80000gn/T/ipykernel_7556/1839780662.py:5: DtypeWarning: Columns (23,51,57,58,59,60,62,72) have mixed types. Specify dtype option on import or set low_memory=False.
  variants = pd.read_csv(variant_path)


In [89]:
variants_subset = variants[[
    'cellosaurus_ids.accession',
    'HugoSymbol',
 #   'Chrom',
    'DNAChange',
    'Pos',
 #   'Ref',
 #   'Alt',
 #   'ProteinChange',
 #   'AF',
 #   'VariantType',
 #   'VariantInfo',
 #   'VepImpact',
 #   'VepBiotype',
 #   'RevelScore',
 #   'VepLofTool',
 #   'LikelyLoF',
 #   'OncogeneHighImpact',
 #   'TumorSuppressorHighImpact',
 #   'AMClass',
 #   'AMPathogenicity',
 #   'Hotspot'
]]
variants_subset.head()

,cellosaurus_ids.accession,HugoSymbol,DNAChange,Pos
0,CVCL_2262,CC2D1B,ENST00000284376.8:c.1335T>A,52358457
1,CVCL_2262,PDE4B,ENST00000341517.9:c.1492C>A,66367803
2,CVCL_2262,AGL,ENST00000361915.8:c.392C>T,99862355
3,CVCL_2262,PPM1J,ENST00000309276.11:c.14T>A,112715288
4,CVCL_2262,ADCY10,ENST00000367851.9:c.4093C>A,167823083


In [108]:
all_mutations = set(variants_subset['DNAChange'])

In [110]:
all_mutations_list = list(all_mutations)

In [99]:
len(all_mutations)

638368

In [111]:
filtered = variants_subset[variants_subset['DNAChange'].str.contains(r':n\.', regex=True, na=False)]
print(filtered)
print(any(":n." in change for change in variants_subset['DNAChange'].unique()))

       cellosaurus_ids.accession HugoSymbol  \
142                    CVCL_N588     CASC15   
294                    CVCL_2268    FAM86DP   
375                    CVCL_2270  LINC02930   
414                    CVCL_2270      DLEU1   
529                    CVCL_2270   IL21-AS1   
...                          ...        ...   
715493                 CVCL_2815    CHEK2P1   
716870                       NaN  LINC01749   
716877                       NaN     LNCBRM   
716920                 CVCL_6C44  LINC00615   
716984                 CVCL_6C44  LINC01097   

                                  DNAChange        Pos  
142            ENST00000652081.1:n.459+1G>A   22610641  
294             ENST00000477247.6:n.97-2A>G   75432872  
375            ENST00000659883.1:n.659-2A>G  120824384  
414            ENST00000479420.5:n.458-1G>A   50589474  
529           ENST00000417927.1:n.2798-1G>A  122648100  
...                                     ...        ...  
715493          ENST00000614446.1:n.

In [86]:
counts = variants_subset.groupby('cellosaurus_ids.accession').size()
print(counts)
print(counts.min(), counts.max(), counts.mean(), counts.std())

cellosaurus_ids.accession
CVCL_0001    135
CVCL_0002     88
CVCL_0004    292
CVCL_0005    163
CVCL_0006    106
            ... 
CVCL_YU12    195
CVCL_YU13    162
CVCL_YU14    168
CVCL_YU15    170
CVCL_ZV94    172
Length: 1854, dtype: int64
25 12303 377.77562028047464 725.8685010146571


In [ ]:
variant_dict_by_cell_line = {}
for cell_line, group in variants_subset.groupby('cellosaurus_ids.accession'):
    # Convert group to list of dictionaries, excluding the grouping column
    variant_dict_by_cell_line[cell_line] = group.drop(columns='cellosaurus_ids.accession').to_dict('records')
variant_dict_by_cell_line['CVCL_0030']

In [90]:
lincs_path = '/content/drive/MyDrive/Data Science Final Project/Data/final/signature_response_features_r2_top0.7_final.parquet'
lincs_path = 'data/signature_response_features_r2_top0.7_final.parquet'
lincs = pd.read_parquet(lincs_path, engine='pyarrow')

In [91]:
compound_path = 'data/comp_info.parquet'
compound_info = pd.read_parquet(compound_path, engine='pyarrow')
compound_info[compound_info['target'].notna()]
compound_info = compound_info[['pert_id', 'target']]

In [92]:
lincs = lincs.merge(compound_info, left_on='pert_id', right_on='pert_id')

In [93]:
lincs['mutations'] = lincs['cellosaurus_id'].map(variant_dict_by_cell_line)

In [ ]:
lincs['mutated_genes'] = lincs['mutations'].apply(lambda x: [variant['HugoSymbol'] for variant in x] if isinstance(x, list) else [])
lincs['target_mutated'] = lincs.apply(
    lambda row: row['target'] in row['mutated_genes'], axis=1
)

In [94]:
lincs

,sig_id,pert_id,cellosaurus_id,inchi_key,cell_drug,nearest_dose,responses,cmap_name,smiles,fps_0,...,9988,9989,999,9990,9991,9992,9993,9997,target,mutations
0,ABY001_HT29_XH:BRD-K70511574:2.5:24,BRD-K70511574,CVCL_0320,OCKHRKSTDPOHEN-BQYQJAHWSA-N,CVCL_0320OCKHRKSTDPOHEN-BQYQJAHWSA-N,2.5,0.191586,HMN-214,COc1ccc(cc1)S(=O)(=O)N(C(C)=O)c1ccccc1C=Cc1cc[...,0,...,-0.116750,-1.074700,0.266150,0.312750,-0.311050,0.015850,0.774600,-0.510450,CSF1R,"[{'HugoSymbol': 'PRAMEF12', 'DNAChange': 'ENST..."
1,ABY001_HT29_XH:BRD-K70511574:2.5:24,BRD-K70511574,CVCL_0320,OCKHRKSTDPOHEN-BQYQJAHWSA-N,CVCL_0320OCKHRKSTDPOHEN-BQYQJAHWSA-N,2.5,0.191586,HMN-214,COc1ccc(cc1)S(=O)(=O)N(C(C)=O)c1ccccc1C=Cc1cc[...,0,...,-0.116750,-1.074700,0.266150,0.312750,-0.311050,0.015850,0.774600,-0.510450,FLT1,"[{'HugoSymbol': 'PRAMEF12', 'DNAChange': 'ENST..."
2,ABY001_HT29_XH:BRD-K70511574:2.5:24,BRD-K70511574,CVCL_0320,OCKHRKSTDPOHEN-BQYQJAHWSA-N,CVCL_0320OCKHRKSTDPOHEN-BQYQJAHWSA-N,2.5,0.191586,HMN-214,COc1ccc(cc1)S(=O)(=O)N(C(C)=O)c1ccccc1C=Cc1cc[...,0,...,-0.116750,-1.074700,0.266150,0.312750,-0.311050,0.015850,0.774600,-0.510450,FLT3,"[{'HugoSymbol': 'PRAMEF12', 'DNAChange': 'ENST..."
3,ABY001_HT29_XH:BRD-K70511574:2.5:24,BRD-K70511574,CVCL_0320,OCKHRKSTDPOHEN-BQYQJAHWSA-N,CVCL_0320OCKHRKSTDPOHEN-BQYQJAHWSA-N,2.5,0.191586,HMN-214,COc1ccc(cc1)S(=O)(=O)N(C(C)=O)c1ccccc1C=Cc1cc[...,0,...,-0.116750,-1.074700,0.266150,0.312750,-0.311050,0.015850,0.774600,-0.510450,FLT4,"[{'HugoSymbol': 'PRAMEF12', 'DNAChange': 'ENST..."
4,ABY001_HT29_XH:BRD-K70511574:2.5:24,BRD-K70511574,CVCL_0320,OCKHRKSTDPOHEN-BQYQJAHWSA-N,CVCL_0320OCKHRKSTDPOHEN-BQYQJAHWSA-N,2.5,0.191586,HMN-214,COc1ccc(cc1)S(=O)(=O)N(C(C)=O)c1ccccc1C=Cc1cc[...,0,...,-0.116750,-1.074700,0.266150,0.312750,-0.311050,0.015850,0.774600,-0.510450,KDR,"[{'HugoSymbol': 'PRAMEF12', 'DNAChange': 'ENST..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182141,DOSVAL005_A375_24H:BRD-A19037878:5,BRD-A19037878,CVCL_0132,RTKIYFITIVXBLE-WKWSCTOISA-N,CVCL_0132RTKIYFITIVXBLE-WKWSCTOISA-N,4.0,0.228885,BRD-A19037878,CC(C=C(C)C=CC(=O)NO)C(=O)c1ccc(cc1)N(C)C,0,...,0.085574,-3.965151,2.360926,1.487882,-1.511580,3.141681,5.463110,0.673212,None,"[{'HugoSymbol': 'SPEN', 'DNAChange': 'ENST0000..."
182142,DOSVAL001_A549_24H:BRD-K19540840:5,BRD-K19540840,CVCL_0023,OUKYUETWWIPKQR-UHFFFAOYSA-N,CVCL_0023OUKYUETWWIPKQR-UHFFFAOYSA-N,4.0,0.706393,saracatinib,CN1CCN(CCOc2cc(OC3CCOCC3)c3c(Nc4c5OCOc5ccc4Cl)...,0,...,-0.334990,-0.230673,-0.650040,-1.271203,-0.336669,-0.480971,-0.794008,-0.868529,ABL1,"[{'HugoSymbol': 'ENO1', 'DNAChange': 'ENST0000..."
182143,DOSVAL001_A549_24H:BRD-K19540840:5,BRD-K19540840,CVCL_0023,OUKYUETWWIPKQR-UHFFFAOYSA-N,CVCL_0023OUKYUETWWIPKQR-UHFFFAOYSA-N,4.0,0.706393,saracatinib,CN1CCN(CCOc2cc(OC3CCOCC3)c3c(Nc4c5OCOc5ccc4Cl)...,0,...,-0.334990,-0.230673,-0.650040,-1.271203,-0.336669,-0.480971,-0.794008,-0.868529,SRC,"[{'HugoSymbol': 'ENO1', 'DNAChange': 'ENST0000..."
182144,DOSVAL006_A549_24H:BRD-K88378636:10,BRD-K88378636,CVCL_0023,DBRXOUCRJQVYJQ-CKNDUULBSA-N,CVCL_0023DBRXOUCRJQVYJQ-CKNDUULBSA-N,10.0,0.160186,withaferin-a,C[C@@H]([C@H]1CC[C@H]2[C@@H]3C[C@H]4O[C@]45[C@...,0,...,-1.487008,-1.915593,-1.596961,0.518605,-2.138142,1.708408,0.332337,2.292674,NAMPT,"[{'HugoSymbol': 'ENO1', 'DNAChange': 'ENST0000..."


In [117]:
lincs.shape

(182146, 18587)

In [118]:
n_rows = 182146
df = pd.DataFrame({mut: [0]*n_rows for mut in all_mutations_list})

: 

In [103]:
lincs['mutations'] = lincs['mutations'].apply(lambda x: x if isinstance(x, list) else [])
for mut in all_mutations:
    lincs[mut] = lincs['mutations'].apply(
        lambda muts: int(any(m['DNAChange'] == mut for m in muts))
    )

KeyboardInterrupt: 

In [106]:
from sklearn.preprocessing import MultiLabelBinarizer

lincs['mutations'] = lincs['mutations'].apply(lambda x: x if isinstance(x, list) else [])
# Extract DNAChange for each row
mutation_lists = lincs['mutations'].apply(
    lambda muts: [m['DNAChange'] for m in muts if 'DNAChange' in m]
)

In [ ]:
mutation_lists_flattened = [item for sublist in mutation_lists for item in sublist]
mutation_unique = 

0         [ENST00000357726.5:c.938C>T, ENST00000375759.8...
1         [ENST00000357726.5:c.938C>T, ENST00000375759.8...
2         [ENST00000357726.5:c.938C>T, ENST00000375759.8...
3         [ENST00000357726.5:c.938C>T, ENST00000375759.8...
4         [ENST00000357726.5:c.938C>T, ENST00000375759.8...
                                ...                        
182141    [ENST00000375759.8:c.7947_7967del, ENST0000040...
182142    [ENST00000234590.10:c.795T>G, ENST00000377205....
182143    [ENST00000234590.10:c.795T>G, ENST00000377205....
182144    [ENST00000234590.10:c.795T>G, ENST00000377205....
182145    [ENST00000234590.10:c.795T>G, ENST00000377205....
Name: mutations, Length: 182146, dtype: object

In [ ]:

mlb = MultiLabelBinarizer()
mutation_matrix = mlb.fit_transform(mutation_lists)

# Create a DataFrame with the binary matrix
mutation_df = pd.DataFrame(
    mutation_matrix, columns=mlb.classes_, index=lincs.index
)

# Concatenate with the original DataFrame if needed
#lincs = pd.concat([lincs, mutation_df], axis=1)


TypeError: '<' not supported between instances of 'float' and 'str'

In [19]:
lincs.to_parquet(path='data/lincs_top0.7_variants.parquet', engine='pyarrow')